In [1]:
import pandas as pd
import time
import threading
from kafkaconsumer import Kafka_Consumer

In [2]:
stop_flag = False

In [3]:
def get_env_params():
    return pd.read_csv('./env_params/env_params.csv',index_col=0, header=None).squeeze().to_dict()

In [4]:
if __name__ == "__main__":
    
    params = get_env_params()

    topics = params['topics'].split('|')


    kafkaconsumers = []
    for iter,topic in enumerate(topics):
        #Create objects of kafka consumer
        kafkaconsumers.append(Kafka_Consumer(topic,[params['bootstrapservers']],str(iter)))
  
    threads = []
    for consumer in kafkaconsumers:
        # Create threads for each consumer function
        thread = threading.Thread(target=consumer.consume_and_upload
                               ,args=(params['adls_account_name'], params['adls_sas_token']
                                      ,int(params['kafkabatchsize']),params['adls_container']))
        threads.append(thread)
        thread.start()

    try:
        while not stop_flag:
            time.sleep(0.1)  # Main thread does something here, could be anything
    except KeyboardInterrupt:
        print("\nKeyboard interruption detected. Stopping all threads...")
        stop_flag = True  # Set the flag to stop threads

    finally:
        # Wait for all threads to complete

        for consumer in kafkaconsumers:
            consumer.consumer.close()
        
        for thread in threads:
            thread.join()
        print("All threads have been stopped.")

current_batch_size 0 Received message: [ConsumerRecord(topic='orders', partition=0, offset=702, timestamp=1726068348690, timestamp_type=0, key=None, value='{"OrderID": "e443df78-9558-467f-9ba9-1faf7a024204", "OrderDate": "2024-09-11 20:47:30", "StoreID": 21, "GlobalID": "Ind-21", "Country": "India", "PaymentMethod": "Cash", "TotalAmount": 825}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=215, serialized_header_size=-1), ConsumerRecord(topic='orders', partition=0, offset=703, timestamp=1726068369676, timestamp_type=0, key=None, value='{"OrderID": "148b2758-d7ab-4928-89e4-69e6ec62b2c8", "OrderDate": "2024-09-11 20:52:30", "StoreID": 17, "GlobalID": "UK-17", "Country": "UK", "PaymentMethod": "Credit Card", "TotalAmount": 535}', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=218, serialized_header_size=-1)]


Exception in thread Thread-7:
Traceback (most recent call last):
  File "c:\Users\Yash\anaconda3\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\Yash\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Yash\Learning\TheBurgerHouse\kafkaconsumer.py", line 61, in consume_and_upload
    file_client.append_data(records, offset=filesize_previous, length=len(records))
  File "c:\Users\Yash\anaconda3\lib\site-packages\azure\core\tracing\decorator.py", line 78, in wrapper_use_tracer
    return func(*args, **kwargs)
  File "c:\Users\Yash\anaconda3\lib\site-packages\azure\storage\filedatalake\_data_lake_file_client.py", line 581, in append_data
    process_storage_error(error)
  File "c:\Users\Yash\anaconda3\lib\site-packages\azure\storage\filedatalake\_deserialize.py", line 221, in process_storage_error
    exec("raise error from None")   # pylint: disable=exec-used # nosec
  File "<string>", line 1, in <m


Keyboard interruption detected. Stopping all threads...
All threads have been stopped.
